In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
import dicom # for reading dicom files
import os # for doing directory operations 
import cv2
import numpy as np
import scipy.ndimage as ndi
import skimage
from skimage.feature import greycomatrix, greycoprops
from skimage import data

In [3]:
def glcm_EH(image):
    glcm = greycomatrix(new_image,distances=[7],angles=[0],levels=256, symmetric=True, normed=True)
    homogenity = greycoprops(glcm, 'homogeneity')[0, 0]
    energy= greycoprops(glcm, 'energy')[0, 0]
    correlation = greycoprops(glcm, 'correlation')[0,0]
    dissimilar = greycoprops(glcm, 'dissimilarity')[0,0]
    return homogenity, energy, correlation, dissimilar


data_dir = 'C:/Users/annam/Desktop/ML/ML - Project/stage1/'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('C:/Users/annam/Desktop/ML/ML - Project/stage1_labels.csv', index_col=0)
IMG_PX_SIZE = 150
new_image = []
df_patient = pd.DataFrame(columns=('patient','hg','eg', 'co', 'di'))
for patient in patients[:2]: # change 2 to len(patients) to run for all patients
    #label = labels_df.get_value(patient, 'cancer')
    path = data_dir + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    #print(slices[0].pixel_array.shape)
    df = pd.DataFrame(columns=('hg','eg', 'co', 'di'))
    for num,each_slice in enumerate(slices[:len(slices)]):
        new_image = np.array(cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)))
        new_image+=2000
        new_image = new_image/64
        new_image = new_image.astype(np.int8)
        new_image = new_image.clip(min=0)
        #segmented_ct_scan = get_segmented_lungs(new_image)
        #segmented_ct_scan
        hg, eg, co, di = glcm_EH(new_image)
        lst = [[hg,eg, co,di]]
        df = df.append(pd.DataFrame(lst, columns=['hg','eg', 'co', 'di']),ignore_index=True)
    hg, eg, co, di = list(df.mean())
    lst_patient = [[patient, hg,eg,co,di]]
    df_patient = df_patient.append(pd.DataFrame(lst_patient, columns=['patient','hg','eg', 'co', 'di']),ignore_index=True)

In [4]:
df_patient

,patient,hg,eg,co,di
0,0015ceb851d7251b8f399e39779d1e7d,0.583047,0.254195,0.798504,4.846441
1,0030a160d58723ff36d73f41b170ec21,0.523532,0.197310,0.764674,5.165976


In [128]:
df_patient.to_csv('GLCM_features_lungs.csv')